In [14]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as Data
import torch.nn as nn
import lmdb

from torch.autograd import Variable
from warpctc_pytorch import CTCLoss
import sys

from torch.utils.data import Dataset
import torchvision.transforms as transforms
import six
import sys
from PIL import Image

import dataset
import utils

import models.crnn as crnn


In [15]:
batchSize=1

In [16]:
model = crnn.CRNN(32, 1, 19, 256)

In [17]:
model

CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace)
    (pooling2): MaxPool2d(kernel_size=(2, 2), stride=(2, 1), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, trac

In [18]:
outputPath ='/Users/chienan/Job/CNN/lmdb/'

train_dataset = dataset.lmdbDataset(root=outputPath)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize,shuffle=True,num_workers=4,
                                           collate_fn=dataset.alignCollate(imgH=32, imgW=320))

In [19]:
alphabet = '零壹貳參肆伍陸柒捌玖拾佰仟萬億兆元整'

nclass = len(alphabet) + 1
nc = 1
criterion = CTCLoss()

converter = utils.strLabelConverter(alphabet)

In [20]:
image = torch.FloatTensor(batchSize, 1, 32, 32)
text = torch.IntTensor(batchSize * 5)
length = torch.IntTensor(batchSize)
image = Variable(image)
text = Variable(text)
length = Variable(length)

In [21]:
# loss averager
loss_avg = utils.averager()

optimizer = optim.Adam(model.parameters(), lr=0.01126,betas=(0.5, 0.999))

In [22]:
def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    utils.loadData(image, cpu_images)
    t, l = converter.encode(cpu_texts[0])
    utils.loadData(text, t)
    utils.loadData(length, l)

    preds = model(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    model.zero_grad()
    cost.backward()
    optimizer.step()
    return cost

In [25]:
total_iteration = 10
for iteration in range(total_iteration):
    train_iter = iter(train_loader)
    i = 0
    while i < len(train_loader):
        for p in model.parameters():
            p.requires_grad = True
        model.train()
        cost = trainBatch(model, criterion, optimizer)
        i += 1
        loss_avg.add(cost)
        
        

        if i % 10 == 0:
            print('[iteration%d/%d][%d/%d] Loss: %f' %
                  (iteration+1, total_iteration, i, len(train_loader), loss_avg.val() ))
            loss_avg.reset()

[iteration0/10][10/10] Loss: 22.434511
[iteration1/10][10/10] Loss: 20.833698
[iteration2/10][10/10] Loss: 24.619755
[iteration3/10][10/10] Loss: 23.076221
[iteration4/10][10/10] Loss: 23.964804
[iteration5/10][10/10] Loss: 43.349091
[iteration6/10][10/10] Loss: 37.575027
[iteration7/10][10/10] Loss: 27.637569
[iteration8/10][10/10] Loss: 24.057203
[iteration9/10][10/10] Loss: 19.811234
